In [32]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 


In [33]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler
import logging
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
import statistics
import torch.optim.lr_scheduler as lr_scheduler

In [34]:
logging.getLogger('mne').setLevel(logging.WARNING)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import import_ipynb
#from Model import net
from Dataloader2 import EEGDataset

In [35]:
def Accuracy(y_pred, y, train_count):
    max_values, _ = torch.max(y_pred, dim=1, keepdim=True)
    mask = y_pred == max_values
    y_pred = mask.int()
    correct_num = torch.sum(torch.all(torch.eq(y, y_pred), dim=1)).item()
    accuracy = correct_num / train_count
    return accuracy * 100

In [36]:
def test(model):
    

    x_test = mne.read_epochs(r"D:\TEST\alahyari.fif", preload=False).get_data(picks='eeg');
    normals = []
    scaler = StandardScaler()
    for idx in range(len(x_test)):
        normals.append(scaler.fit_transform(x_test[idx]))
    normals = torch.tensor(normals).cuda().float()
    result = torch.argmax(model(normals), axis=1)
    unique_elements, counts = torch.unique(result, return_counts=True)
    return (counts[0] / result.shape[0])


In [37]:
class net(nn.Module):
    def __init__(self, T, C, input_size, hidden_size, num_layers, spatial_num, dropout, pool):
        super(net, self).__init__()
        
        self.T = T
        self.C = C
        self.spatial_num = spatial_num
        self.dropout = dropout
        self.pool = pool

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.cell_count = self.T // self.input_size

        self.fcn_in = (spatial_num * self.hidden_size)

        self._lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)

        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])

        self.cnn_block = nn.Sequential(nn.Conv2d(1, self.spatial_num, (self.C, 1)),
                                       nn.BatchNorm2d(self.spatial_num),
                                       nn.ELU(),



                                       nn.Dropout(self.dropout))

        
        self.fcn = nn.Sequential(nn.Linear(self.fcn_in, 128), 
                                 nn.ReLU(),
                                 nn.Linear(128, 16),
                                 nn.Dropout(self.dropout),
                                 nn.ReLU(),
                                 nn.Linear(16, 4))

        #self.fcn = nn.Linear(self.fcn_in, 4)
        self.results = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.reshape(-1, 1, 19, 3000)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.cell_count, self.input_size)
        _x = None

        for index, cell in enumerate(self.lstm):
            cell_out, _ = cell(x[:, index, :, :], None)
            last_layer_out = cell_out[:, -1, :]
            
            last_layer_out = last_layer_out.unsqueeze(0)
            if _x is None:
                _x = last_layer_out
            else:
                _x = torch.cat((_x, last_layer_out), dim=0)
            

        x = _x.permute(1, 0, 2).unsqueeze(1)

        x = self.cnn_block(x)


        x = x.reshape(self.N, -1)

        x = self.fcn(x)
        x = self.results(x)

        return x

In [38]:
MNE_Data = EEGDataset(root_dir=r"C:\Users\admin\Desktop\MNE Data")
#MNE_Data = EEGDataset(root_dir=r"D:\TEST MNE")


In [39]:
batch_size = 1
train_dataloader = DataLoader(MNE_Data, batch_size=batch_size, shuffle=True)

In [40]:
model = net(T = 3000, C = 19, input_size = 300, hidden_size = 100, num_layers=1, spatial_num= 30, dropout=0.5, pool=1).to(device)

In [41]:
criterion = nn.CrossEntropyLoss(weight = torch.Tensor([5.3125, 2.8333, 3.5417, 5.6667]).cuda())

optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.Adagrad(model.parameters(), lr=0.001)


scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.1)
epochs = 60

In [42]:

for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        log = []
        for index, data in enumerate(train_dataloader):
            
            x, y = data
            y = y.to(torch.float64)
            x = x.reshape(-1, 1, 19, 3000).float()
            #x = x[torch.randperm(x.shape[0])]
            y = F.one_hot(torch.tensor(torch.tensor([y.item()]).to(torch.int64)), num_classes=4).expand(x.shape[0], -1).float()
            
            train_count = x.shape[0]
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #scheduler.step()
            log.append(Accuracy(y_pred, y, train_count))
            torch.cuda.empty_cache()
        print(f"test accuracy: {test(model)}")
        
        print(f"mean train accuracy across epoch: {statistics.mean(log)}")
        
        #print(f"Train Accuracy:{correct_num / train_count}")
        """
        """

  2%|▏         | 1/60 [00:45<45:01, 45.79s/it]

test accuracy: 0.009259259328246117
mean train accuracy across epoch: 27.315487114551193


  3%|▎         | 2/60 [01:57<58:57, 60.99s/it]

test accuracy: 0.02777777798473835
mean train accuracy across epoch: 21.966555555150496


  5%|▌         | 3/60 [03:12<1:03:53, 67.26s/it]

...

test accuracy: 0.7222222089767456
mean train accuracy across epoch: 85.25385943133884


 98%|█████████▊| 59/60 [47:30<00:46, 46.15s/it]

test accuracy: 0.5601851940155029
mean train accuracy across epoch: 86.16468490134342


100%|██████████| 60/60 [48:16<00:00, 48.28s/it]

test accuracy: 0.5833333134651184
mean train accuracy across epoch: 85.62395206928481


In [43]:
torch.argmax(model(x), axis=1)


tensor([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1,

In [44]:
print(f"test accuracy: {test(model)}")

test accuracy: 0.5416666865348816


In [45]:
"""
for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        log = []
        for index, data in enumerate(train_dataloader):
            
            x, y = data
            y = y.to(torch.float64)
            print(y)
            
            x = x.reshape(-1, 1, 19, 3000).float()
            y = torch.tensor([y.item()]).to(torch.int64).expand(x.shape[0], -1).float()
            
            
            train_count = x.shape[0]
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            print(y[-1])
            print(Accuracy(y_pred, y, train_count))
            log.append(Accuracy(y_pred, y, train_count))
            torch.cuda.empty_cache()
            
        
        """
        #print(f"mean accuracy across epoch: {statistics.mean(log)}")
        
        #print(f"Train Accuracy:{correct_num / train_count}")

'\nfor epoch in trange(epochs):\n        model.train()\n        running_loss = 0.0\n        correct_num = 0\n        log = []\n        for index, data in enumerate(train_dataloader):\n            \n            x, y = data\n            y = y.to(torch.float64)\n            print(y)\n            \n            x = x.reshape(-1, 1, 19, 3000).float()\n            y = torch.tensor([y.item()]).to(torch.int64).expand(x.shape[0], -1).float()\n            \n            \n            train_count = x.shape[0]\n            x, y = x.to(device), y.to(device)\n            y_pred = model(x)\n            loss = criterion(y_pred, y)\n            optimizer.zero_grad()\n            loss.backward()\n            optimizer.step()\n            scheduler.step()\n            print(y[-1])\n            print(Accuracy(y_pred, y, train_count))\n            log.append(Accuracy(y_pred, y, train_count))\n            torch.cuda.empty_cache()\n            \n        \n        '

In [46]:
file_path = 'D:\model weights\model_weights.pth'
    torch.save(model.state_dict(), file_path)
    model.load_state_dict(torch.load(file_path))

IndentationError: unexpected indent (3638288533.py, line 2)

In [ ]:
"""
y = F.one_hot(torch.randint(low=0, high=4, size=(1,)), 4).float()
yhat = torch.rand((1, 4)).float()
criterion(y, yhat)
"""

'\ny = F.one_hot(torch.randint(low=0, high=4, size=(1,)), 4).float()\nyhat = torch.rand((1, 4)).float()\ncriterion(y, yhat)\n'

In [ ]:
"""
__ = (torch.round(y_pred).to(torch.int64) == y)
preds = np.all(__.cpu().numpy(), axis=1)
correct_num += np.count_nonzero(preds)
"""

'\n__ = (torch.round(y_pred).to(torch.int64) == y)\npreds = np.all(__.cpu().numpy(), axis=1)\ncorrect_num += np.count_nonzero(preds)\n'

<All keys matched successfully>

tensor(0.4861, device='cuda:0')